<a href="https://colab.research.google.com/github/normalnon/Advanced-Data-Filtering/blob/main/Workshop2_PPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Get your Hugging Face Token

If you don't already have one, create a **User Access Token** on the Hugging Face website. Make sure it has at least 'read' permissions.

- Go to [huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)
- Click on 'New token'
- Give it a name (e.g., `Colab_HF_Token`)
- Set the Role to 'read' (or 'write' if you plan to push models)
- Click 'Generate a token' and copy the token.

## Add the token to Colab Secrets

To securely store your token and avoid hardcoding it in your notebook:

- Click on the '🔑' (key) icon in the left panel of Colab.
- Click on 'Add new secret'.
- For the 'Name' field, type `HF_TOKEN` (this is the recommended name by Hugging Face).
- For the 'Value' field, paste your Hugging Face token.
- Ensure 'Notebook access' is toggled on for this notebook.


## Log in to Hugging Face programmatically

Now, run the following code cell to log in using the token stored in your Colab secrets. This will authenticate your session and allow you to access gated models.

In [ ]:
# Install huggingface_hub if not already installed
!pip install -q huggingface_hub

from huggingface_hub import login
from google.colab import userdata

# Retrieve the token from Colab secrets
hf_token = userdata.get('Huggingface')

# Log in to Hugging Face
if hf_token:
    login(token=hf_token)
    print("Successfully logged in to Hugging Face!")
else:
    print("Hugging Face token not found in Colab secrets. Please add it as 'HF_TOKEN'.")


Successfully logged in to Hugging Face!


## ติดตั้ง Library

In [ ]:
# ติดตั้ง Library ที่จำเป็นสำหรับการรัน Model และจัดการข้อมูล
!pip install transformers torch pandas accelerate --quiet

print("Installation complete.")

Installation complete.


## Imports & Setup


In [ ]:
import math
import torch
import pandas as pd
import numpy as np
from typing import List, Tuple
from transformers import AutoTokenizer, AutoModelForCausalLM

# เลือกโมเดล
MODEL_NAME = "Qwen/Qwen3-0.6B"

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
print(f"Selected Model: {MODEL_NAME}")

Using device: cuda
Selected Model: Qwen/Qwen3-0.6B


## สร้าง Mock-up Data

In [ ]:
# Prepare Mock-up Data
def get_mock_data():
    data = {
        "Group": [],
        "Text": []
    }

    # 1. กลุ่มภาษาธรรมชาติ (ควรได้ PPL ต่ำ/ดี)
    normal_texts = [
        "กรุงเทพมหานครเป็นเมืองหลวงของประเทศไทย",
        "แมวชอบนอนอาบแดดในตอนเช้าเพราะอากาศอบอุ่น",
        "การเรียนรู้เครื่องจักรเป็นสาขาหนึ่งของปัญญาประดิษฐ์"
    ]
    for t in normal_texts:
        data["Group"].append("1. Normal (Natural)")
        data["Text"].append(t)

    # 2. กลุ่มสลับคำ/ไวยากรณ์ผิด (ควรได้ PPL สูงขึ้น)
    scrambled_texts = [
        "ของเมืองหลวงเป็นไทยกรุงเทพมหานครประเทศ",
        "เช้าแดดอาบชอบแมวนอนในอบอุ่นอากาศเพราะ",
        "สีเขียวนอนหลับอย่างรวดเร็วบนเพดานความคิด" # Semantic ผิด
    ]
    for t in scrambled_texts:
        data["Group"].append("2. Scrambled/Weird")
        data["Text"].append(t)

    # 3. กลุ่มภาษาผสม/Code-switching (PPL ขึ้นกับความเก่งโมเดล)
    mixed_texts = [
        "วันนี้ Coding สนุกมาก Error กระจาย 555+",
        "Concept ของ Perplexity คือวัดความ Surprise ของ Model",
    ]
    for t in mixed_texts:
        data["Group"].append("3. Code-Switching")
        data["Text"].append(t)

    # 4. กลุ่มมั่ว/สุ่ม (PPL ควรสูงที่สุด)
    gibberish_texts = [
        "กขคง จฉชซ 1234 abcd !@#$",
        "sfdk jljlaksjd 9999 xxxxx"
    ]
    for t in gibberish_texts:
        data["Group"].append("4. Gibberish")
        data["Text"].append(t)

    return pd.DataFrame(data)

# สร้างและแสดงตัวอย่างข้อมูล
df_data = get_mock_data()
display(df_data)

,Group,Text
0,1. Normal (Natural),กรุงเทพมหานครเป็นเมืองหลวงของประเทศไทย
1,1. Normal (Natural),แมวชอบนอนอาบแดดในตอนเช้าเพราะอากาศอบอุ่น
2,1. Normal (Natural),การเรียนรู้เครื่องจักรเป็นสาขาหนึ่งของปัญญาประ...
3,2. Scrambled/Weird,ของเมืองหลวงเป็นไทยกรุงเทพมหานครประเทศ
4,2. Scrambled/Weird,เช้าแดดอาบชอบแมวนอนในอบอุ่นอากาศเพราะ
5,2. Scrambled/Weird,สีเขียวนอนหลับอย่างรวดเร็วบนเพดานความคิด
6,3. Code-Switching,วันนี้ Coding สนุกมาก Error กระจาย 555+
7,3. Code-Switching,Concept ของ Perplexity คือวัดความ Surprise ของ...
8,4. Gibberish,กขคง จฉชซ 1234 abcd !@#$
9,4. Gibberish,sfdk jljlaksjd 9999 xxxxx


## Define Calculation Functions

In [ ]:
# Define Calculation Functions

def mask_special_tokens(labels: torch.Tensor, special_ids: set[int]) -> torch.Tensor:
    """เปลี่ยน label ของ special token เป็น -100 เพื่อไม่ให้เอามาคิด Loss"""
    masked = labels.clone()
    for tid in special_ids:
        masked[masked == tid] = -100
    return masked

@torch.no_grad()
def calculate_ppl_by_sentence(model, tokenizer, texts: List[str]):
    """
    คำนวณ PPL แยกรายประโยค และส่งค่ากลับมาเพื่อคำนวณ Micro/Macro
    """
    model.eval()
    results = []

    # ดึง ID ของ token พิเศษที่ไม่ควรนำมาคิด loss (เช่น padding)
    special_ids = {
        tokenizer.eos_token_id,
        tokenizer.bos_token_id,
        tokenizer.pad_token_id
    }
    special_ids = {tid for tid in special_ids if tid is not None}

    total_nll_sum = 0.0      # ผลรวม NLL ทั้งหมด (ใช้หา Micro)
    total_token_count = 0    # ผลรวม Token ทั้งหมด (ใช้หา Micro)

    for text in texts:
        # 1. Tokenize
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=1024).to(device)

        # 2. Prepare Labels (Mask special tokens)
        labels = inputs.input_ids.clone()
        labels = mask_special_tokens(labels, special_ids)

        # 3. Forward Pass
        outputs = model(input_ids=inputs.input_ids, attention_mask=inputs.attention_mask, labels=labels)
        loss = outputs.loss.item() # นี่คือค่าเฉลี่ย NLL ของประโยคนี้

        # 4. Count valid tokens
        valid_tokens = (labels != -100).sum().item()

        # 5. Calculate Metrics for this sentence
        sentence_nll_sum = loss * valid_tokens # แปลงค่าเฉลี่ยกลับเป็นผลรวม NLL ของประโยค
        sentence_ppl = math.exp(loss)          # PPL = exp(NLL_mean)

        # เก็บสะสมค่ารวม
        total_nll_sum += sentence_nll_sum
        total_token_count += valid_tokens

        results.append({
            "Token_Count": valid_tokens,
            "NLL_Sum": sentence_nll_sum,
            "Perplexity": sentence_ppl
        })

    return results, total_nll_sum, total_token_count

## Load model & Calcualation

In [ ]:
# Load Model & Run Calculation
print(f"Loading {MODEL_NAME}...")

# โหลด Tokenizer และ Model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto"
)

print("Calculating Perplexity...")

sentence_results, total_nll, total_tokens = calculate_ppl_by_sentence(
    model,
    tokenizer,
    df_data["Text"].tolist()
)

# นำผลลัพธ์มารวมกับ DataFrame เดิม
df_results = pd.concat([df_data, pd.DataFrame(sentence_results)], axis=1)

print("Calculation Done!")

Loading Qwen/Qwen3-0.6B...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Calculating Perplexity...
Calculation Done!


## Result Analysis

In [ ]:
# Result Analysis

# --- 1. Calculate Micro vs Macro ---

# Micro: มองทุกประโยคต่อกันเป็นก้อนเดียว (Weighted by length)
# สูตร: exp( ผลรวม NLL ทั้งหมด / ผลรวม Token ทั้งหมด )
micro_ppl = math.exp(total_nll / total_tokens)

# Macro: มองทุกประโยคสำคัญเท่ากัน (Average of PPLs)
# สูตร: Average( PPL ของแต่ละประโยค )
macro_ppl = df_results["Perplexity"].mean()

print("="*40)
print(f"📊 Overall Results ({MODEL_NAME})")
print("="*40)
print(f"👉 Micro Average PPL:  {micro_ppl:.4f}")
print(f"👉 Macro Average PPL:  {macro_ppl:.4f}")
print("="*40)

# --- 2. Group Analysis ---
print("\n🔍 Average Perplexity by Group (Lower is Better):")
group_summary = df_results.groupby("Group")["Perplexity"].mean().sort_values()
display(group_summary.to_frame())

print("\n📋 Detailed Sentence Report:")
pd.set_option('display.max_colwidth', 60)
pd.set_option('display.float_format', '{:.2f}'.format)
display(df_results[["Group", "Text", "Token_Count", "Perplexity"]])

📊 Overall Results (Qwen/Qwen3-0.6B)
👉 Micro Average PPL:  73.0944
👉 Macro Average PPL:  204.1737

🔍 Average Perplexity by Group (Lower is Better):


,Perplexity
Group,
1. Normal (Natural),17.200837
2. Scrambled/Weird,103.028627
3. Code-Switching,257.745277
4. Gibberish,582.779046



📋 Detailed Sentence Report:


,Group,Text,Token_Count,Perplexity
0,1. Normal (Natural),กรุงเทพมหานครเป็นเมืองหลวงของประเทศไทย,14,13.90
1,1. Normal (Natural),แมวชอบนอนอาบแดดในตอนเช้าเพราะอากาศอบอุ่น,18,32.14
2,1. Normal (Natural),การเรียนรู้เครื่องจักรเป็นสาขาหนึ่งของปัญญาประดิษฐ์,28,5.57
3,2. Scrambled/Weird,ของเมืองหลวงเป็นไทยกรุงเทพมหานครประเทศ,15,35.56
4,2. Scrambled/Weird,เช้าแดดอาบชอบแมวนอนในอบอุ่นอากาศเพราะ,17,217.50
5,2. Scrambled/Weird,สีเขียวนอนหลับอย่างรวดเร็วบนเพดานความคิด,21,56.03
6,3. Code-Switching,วันนี้ Coding สนุกมาก Error กระจาย 555+,18,251.39
7,3. Code-Switching,Concept ของ Perplexity คือวัดความ Surprise ของ Model,16,264.10
8,4. Gibberish,กขคง จฉชซ 1234 abcd !@#$,18,479.15
9,4. Gibberish,sfdk jljlaksjd 9999 xxxxx,14,686.41
